<a href="https://colab.research.google.com/github/Priyansh295/Lab-Portal/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'car-damage-severity-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2774887%2F4793070%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240607%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240607T082719Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D999877f0fc4f371aa49364c89c205c77aa2fe1e403ed23ecbb0c83501a1c3805af1825dcac7373c644e7ab998fddf9fe4287552292f29c5d9829e8867e6b3cc78b584425c13e8d5e846ed0eaeaf9e5264e5fb27a9cb8e63fb910172f88b2a3523b3b549fd20ae6c4bfcf4ac4e8f721bd63b93910ec04a9bfe6476033167f4a31652b72b4e5727e7ccdb34fccd61163aa630fc633c1ba497f2f9a274914dcce9ef6c4bdaebd9cdf0e27fa4ff672e7f22a97033cb77ac50909aac83240255535cf5168fb1f872fb9ea390852a419c24114b0ada15916c8e0e113714677b44d86c08d3c664d5b7e747d9ac5bc441ed3e6f602ba05c4eab173ee39bacfafae955f7f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers



# Data Preparation

In [ ]:
train_img_path = '/kaggle/input/car-damage-severity-dataset/data3a/training'
test_img_path = '/kaggle/input/car-damage-severity-dataset/data3a/validation'

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        validation_split=0.20,)

# Use flow_from_directory for the training dataset
train_ds = train_data_gen.flow_from_directory(
    train_img_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=123,
    shuffle=True
)
valid_ds = train_data_gen.flow_from_directory(
    train_img_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=123,
    shuffle=True
)

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1./255,)  # You may adjust other parameters as needed

# Use flow_from_directory for the test dataset
test_ds = test_data_gen.flow_from_directory(
    test_img_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Assuming label_mode='int' in your original code
    shuffle=False  # Set to True if you want to shuffle the data
)

In [ ]:
cl=test_ds.class_indices
print(cl)

In [ ]:
def plot_images_from_dataset(dataset, num_images=9):
    # Fetch a batch of images and labels from the dataset
    images, labels = next(iter(dataset))

    plt.figure(figsize=(10, 10))
    for i in range(min(num_images, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])

        # Map the label index back to the original class name
        label_index = labels[i].argmax()  # Assumes one-hot encoding
        class_name = next(key for key, value in cl.items() if value == label_index)

        plt.title(f"Class: {class_name}")
        plt.axis("off")
    plt.show()


# Assuming test_ds is your dataset
plot_images_from_dataset(test_ds)


> **CNN USING PRETRAINED EFF NET****
****

A Convolutional Neural Network (CNN) is a type of artificial intelligence model specifically designed for analyzing visual data, such as images. It works by using layers of filters (also called kernels) that automatically learn to detect important features in the images, like edges, textures, and shapes. These filters slide over the image and create feature maps that highlight these detected features, allowing the network to understand and identify various patterns within the image.

CNNs are particularly effective for image-related tasks because they can recognize spatial hierarchies in data. This means they can first identify simple features (like edges) and then combine them to recognize more complex structures (like shapes and objects). As a result, CNNs are widely used for image classification, object detection, and other tasks where understanding visual content is crucial. They help computers achieve tasks like recognizing faces in photos, detecting objects in videos, or even driving cars by interpreting camera inputs.
![image.png](attachment:2488b779-4068-4bff-9b00-7f093dcb4ea0.png)

In [ ]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras import regularizers
from tensorflow_addons.metrics import F1Score

img_size = (224, 224)
lr = 0.001
class_count = 3

img_shape = (img_size[0], img_size[1], 3)

base_model = DenseNet169(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
base_model.trainable = True
x = base_model.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = Dense(256, kernel_regularizer=regularizers.l2(l=0.016),
          activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu')(x)
x = Dropout(rate=.4, seed=123)(x)
output = Dense(class_count, activation='softmax')(x)
model_eff = Model(inputs=base_model.input, outputs=output)
model_eff.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy',
              metrics=['accuracy','AUC'])





This code uses a pre-trained DenseNet169 model, which means it’s a model that has already been trained on a large dataset (ImageNet) and knows how to recognize many common features in images. We exclude the final layers of DenseNet169 to add our own custom layers for our specific task. This allows us to take advantage of the pre-trained model's knowledge and adapt it to classify images into three categories.

The added layers include batch normalization to stabilize and speed up training, a dense layer with regularization to prevent overfitting, dropout to randomly turn off some neurons during training for better generalization, and a final dense layer with softmax activation to output probabilities for each class. Using this pre-trained model helps our new model learn faster and perform better with less training data and time. This approach is known as transfer learning.
![![image.png](attachment:a300db1f-0fdb-4b70-a291-2ce20f11363f.png)]


In [ ]:
epochs=50

# Train the model
history_eff = model_eff.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    verbose=1,
)

# Save training and validation histories for later analysis
all_train_histories = [history_eff.history['accuracy']]
all_val_histories = [history_eff.history['val_accuracy']]

In [ ]:
model_eff.save('model_eff.h5')

When we train a neural network model, we do it in cycles called epochs. Each epoch represents one complete pass through the entire training dataset. During each epoch, the model makes predictions on the training data, calculates errors (differences between predicted and actual values), and adjusts its internal parameters (weights) to reduce these errors. This process of predicting, calculating errors, and updating weights is repeated for a specified number of epochs or until the model's performance stops improving.

Saving weights during training means storing the current values of the model’s parameters. These weights are essentially the learned features that the model has identified as important for making accurate predictions. By saving these weights, we can preserve the state of the model at a particular point in training. This is useful for several reasons: it allows us to resume training later if needed, to prevent losing progress if the training process is interrupted, and to use the trained model for making predictions on new data without retraining it from scratch. Additionally, saving the best weights (those that achieve the highest accuracy on validation data) ensures we have the most effective version of the model for deployment

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Function to preprocess image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict and display the image with its predicted class
def predict_and_display_image(image_path, model):
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(processed_image)

    # Get the predicted class label
    predicted_class_index = np.argmax(predictions)
    class_name = next(key for key, value in cl.items() if value == predicted_class_index)

    # Display the image with its predicted class
    img = load_img(image_path)
    plt.imshow(img)
    plt.title(f"Predicted Class: {class_name}")
    plt.axis("off")
    plt.show()

# Path to your own image
your_image_path = '/kaggle/input/test-2-minor/0015.JPEG'  # Change this to your image path

# Predict and display the image
predict_and_display_image(your_image_path, model_eff)


In [ ]:
!pip install ipywidgets


> **Try uploading pictures of damaged vehicles and see whether those damages are minor, major or severe**


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt

# Function to preprocess image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict and display the image with its predicted class
def predict_and_display_image(image_path, model):
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(processed_image)

    # Get the predicted class label
    predicted_class_index = np.argmax(predictions)
    class_name = next(key for key, value in cl.items() if value == predicted_class_index)

    # Display the image with its predicted class
    img = load_img(image_path)
    plt.imshow(img)
    plt.title(f"Predicted Class: {class_name}")
    plt.axis("off")
    plt.show()

# Define the upload button and output widget
upload_btn = widgets.FileUpload(accept='image/*', multiple=False)
output = widgets.Output()

# Function to handle the image upload and classification
def on_upload_change(change):
    for filename, file_info in change['new'].items():
        # Save the uploaded image to a temporary path
        with open('uploaded_image.jpg', 'wb') as f:
            f.write(file_info['content'])

        # Display the image and prediction
        with output:
            output.clear_output()  # Clear previous outputs
            predict_and_display_image('uploaded_image.jpg', model_eff)

# Attach the function to the upload button
upload_btn.observe(on_upload_change, names='value')

# Display the upload button and output widget
display(upload_btn, output)


In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_and_display_image(image_path, model):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    predicted_class_index = np.argmax(predictions)
    class_name = next(key for key, value in cl.items() if value == predicted_class_index)
    img = load_img(image_path)
    plt.imshow(img)
    plt.title(f"Predicted Class: {class_name}")
    plt.axis("off")
    plt.show()

upload_btn = widgets.FileUpload(accept='image/*', multiple=False)
output = widgets.Output()

def on_upload_change(change):
    for filename, file_info in change['new'].items():
        with open('uploaded_image.jpg', 'wb') as f:
            f.write(file_info['content'])
        with output:
            output.clear_output()
            predict_and_display_image('uploaded_image.jpg', model_eff)

upload_btn.observe(on_upload_change, names='value')
display(upload_btn, output)
